# 1. Instalación de dependencias e importación de paquetes

In [146]:
# Install dependencies
!pip install pandas
!pip install numpy
!pip install -U scikit-learn

In [147]:
# Import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# 2. Preprocesamiento del *dataset*: [Room Occupancy Estimation Data Set](https://www.kaggle.com/ananthr1/room-occupancy-estimation-data-set)

In [148]:
# Load data as dataframe
df = pd.read_csv('Occupancy_Estimation.csv')
df.shape

(10129, 19)

In [149]:
df.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 1: Recodificación de la columna *RoomOccupancyCount*

En este caso, la columna que contiene la variable de salida (*RoomOccupancyCount*) cuenta el número de personas en la sala. Para simplificar, nuestro objetivo va a ser detectar si la sala está vacía o, en cambio, hay alguien (no el número exacto de personas).

Por lo tanto, se **recodificará cualquier valor mayor que 0 en esa columna como 1**, de tal forma que la variable de salida sea binaria:
- 0 -> ausencia.
- 1 -> presencia.

In [150]:
# Show "Room_Occupancy_Count" before processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

In [151]:
#Process "Room_Occupancy_Count"
df["Room_Occupancy_Count"] = np.where(df["Room_Occupancy_Count"] > 0, 1, 0)

# Show "Room_Occupancy_Count" after processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int32

## Paso 2: Eliminación de las columnas temporales *Date* y *Time*

Respecto a los datos ofrecidos por las columas *Date* y *Time*, vamos a filtrarlos y dejarlos fuera del proceso. Ya que, aunque los incluyeramos (por ejemplo, juntando ambas partes en una sola columna y convirtiéndolo a formato 'epoch', como long int), los valores de muestras sucesivas de esa columna estarían totalmente correlados entre sí y, como consecuencia, fastidiaríamos a la mayoría de algoritmos que vamos a emplear posteriormente.

Se debe tener en cuenta que no nos estamos olvidando de la información temporal para considerar como si cada valor muestreado (fila de la tabla) fuese independiente de las demás filas. Sabemos que eso no es así, pero estamos considerando que esa información temporal no nos ofrece valor añadido para predecir si la habitación está ocupada o vacía. En caso contrario, se tendrían que usar modelos bastante más complicados para considerar esa relación temporal que indica que, en realidad, las muestras de dos filas adyacentes son consecutivas en el tiempo.


In [152]:
# Filter Date and Time columns
df.drop(['Time'], axis=1, inplace=True)
df.drop(['Date'], axis=1, inplace=True)

# Show the result
df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 3: División del *dataset* en *train* y *test*

In [153]:
# Split dataset into train and test
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((8103, 17), (2026, 17))

In [154]:
train_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
5453,25.06,25.13,24.75,25.69,10,11,48,33,0.08,0.04,0.06,0.11,355,0.000000,0,0,0
993,26.13,26.44,25.81,26.25,0,0,0,0,0.07,0.05,0.05,0.06,1075,-4.300000,0,0,0
6062,25.44,25.50,25.38,26.13,9,10,45,33,0.07,0.06,0.07,0.10,355,0.000000,0,0,0
7104,25.38,25.38,24.94,25.81,0,0,0,0,0.08,0.06,0.06,0.09,355,0.000000,0,0,0
1148,25.69,25.75,25.38,25.88,0,0,0,0,0.07,0.05,0.05,0.06,655,-2.434615,0,0,0


In [155]:
test_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
9954,25.06,25.06,24.56,25.13,1,1,10,5,0.09,0.05,0.06,0.09,345,0.000000,0,0,0
9429,25.19,25.19,24.69,25.25,0,0,0,0,0.07,0.04,0.06,0.09,345,0.000000,0,0,0
224,25.94,28.19,25.50,26.13,162,247,228,70,0.79,0.81,1.36,1.84,835,8.096154,1,0,1
927,26.31,28.25,26.13,26.44,150,238,181,10,1.18,0.63,1.95,0.50,1210,-0.407692,1,1,1
1503,25.38,25.44,24.94,25.75,0,0,0,0,0.07,0.05,0.07,0.06,375,-0.207692,0,0,0


# 3. Aplicación del modelo *LogisticRegression*

## Paso 1: Entrenamiento
Para el caso de este modelo, primero debemos separar las columnas con las características (*features*) de la columna que contiene la variable de salida (*Room_Occupancy_Count*):

In [156]:
# Get output column
train_output = train_df['Room_Occupancy_Count']
# Get features columns
train_features = train_df.copy()
train_features.drop(['Room_Occupancy_Count'], axis=1, inplace=True)

In [157]:
train_features.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR
5453,25.06,25.13,24.75,25.69,10,11,48,33,0.08,0.04,0.06,0.11,355,0.000000,0,0
993,26.13,26.44,25.81,26.25,0,0,0,0,0.07,0.05,0.05,0.06,1075,-4.300000,0,0
6062,25.44,25.50,25.38,26.13,9,10,45,33,0.07,0.06,0.07,0.10,355,0.000000,0,0
7104,25.38,25.38,24.94,25.81,0,0,0,0,0.08,0.06,0.06,0.09,355,0.000000,0,0
1148,25.69,25.75,25.38,25.88,0,0,0,0,0.07,0.05,0.05,0.06,655,-2.434615,0,0


In [158]:
train_output.head()

5453    0
993     0
6062    0
7104    0
1148    0
Name: Room_Occupancy_Count, dtype: int32

In [159]:
#Scaling features data
scaler = StandardScaler()
scaled_train_features = scaler.fit_transform(train_features)

Una vez realizada la separación, procedemos a ejecutar el entrenamiento del modelo:

In [160]:
model = LogisticRegression()
model.fit(scaled_train_features, train_output)

LogisticRegression()

## Paso 2: Inferencia

Probamos el modelo entrenado, utilizando el conjunto de test:

In [161]:
# Get feature columns
test_df.drop(['Room_Occupancy_Count'], axis=1, inplace=True)
# Inference
model.predict(test_df.values)

array([0, 0, 0, ..., 0, 0, 0])